### Breast Cancer CNN

In [2]:
from sklearn.datasets import load_breast_cancer
import numpy as np

breast_cancer = load_breast_cancer()
breast_cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [3]:
breast_cancer['data'].shape

(569, 30)

In [4]:
breast_cancer['feature_names']

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [5]:
breast_cancer['target_names']

array(['malignant', 'benign'], dtype='<U9')

### Use 30% of the data as labeled data

In [6]:
data, labels = breast_cancer.data[:,:], breast_cancer.target[:]

num_samples = len(labels)
shuffle_order = np.random.permutation(num_samples)

data = data[shuffle_order, :]
labels = labels[shuffle_order]

data = data / np.amax(data, axis = 0)
labels = labels / np.amax(labels, axis = 0)
data = data.astype('float32')
labels = labels.astype('float32')

print(data.shape)

(569, 30)


In [7]:
X, y = data, labels

train_split = 0.6
n_train = int(train_split * num_samples)
x_train, y_train = X[:n_train], y[:n_train]
x_test, y_test = X[n_train:], y[n_train:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(341, 30)
(341,)
(228, 30)
(228,)


In [8]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['KERAS_BACKEND'] = 'tensorflow'

import matplotlib.pyplot as plt
import numpy as np
import random

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import Activation

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
num_classes = 2

#reshape data
x_train = x_train.reshape((341,30,1)) / 255.
x_test = x_test.reshape((228,30,1)) / 255.

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('%d train samples, %d test samples'%(x_train.shape[0], x_test.shape[0]))
print('training data shape: ', x_train.shape, y_train.shape)
print('test data shape: ', x_test.shape, y_test.shape)

341 train samples, 228 test samples
training data shape:  (341, 30, 1) (341, 2)
test data shape:  (228, 30, 1) (228, 2)


In [10]:
x_train.shape[1:]

(30, 1)

In [11]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
# model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 28, 64)            256       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 64)            12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 6, 128)            24704     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 128)            49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 512)            66048     
__________

In [12]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)

ValueError: Error when checking target: expected dense_2 to have 3 dimensions, but got array with shape (341, 2)